In [68]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import re
import matplotlib.pyplot as plt

In [69]:
from IPython.display import display

In [70]:
pd.options.display.max_columns = None
pd.options.display.max_rows = 100

### Load HDF5 file from working directory 

In [71]:
os.chdir(r'C:\Users\Sellit\Desktop\BA_Coding\Data\1final_out')

In [72]:
start4=datetime.now()

df_out = pd.read_hdf('DataFrame_1_df_out.h5', 'df_out')

end4 = datetime.now()
print(end4 - start4)

0:00:04.325028


#### only first 5 pages

In [73]:
df_out = df_out.groupby('session_id').head(5)

### Check out dataframe

In [74]:
df_out = df_out.reset_index(drop=True)

In [75]:
df_out.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48800 entries, 0 to 48799
Data columns (total 44 columns):
idx                        48800 non-null int32
purchase                   48800 non-null bool
session_id                 48800 non-null object
num_pages                  48800 non-null int64
created                    48800 non-null datetime64[ns]
lastActivity               48800 non-null datetime64[ns]
startTime                  48800 non-null datetime64[ns]
endTime                    48800 non-null datetime64[ns]
engagementTime             48800 non-null float64
visitTime                  48800 non-null float64
total_engagementTime       48800 non-null float64
duration                   48800 non-null float64
page_cat                   48800 non-null object
scroll                     48800 non-null float64
uri                        48800 non-null object
websitePage                48800 non-null object
ref_gen                    31015 non-null object
page_id                  

### Add features about mouse movement behavior

#### Load DataFrame 2

In [76]:
start4=datetime.now()

df_purch = pd.read_hdf('df2_for_merge_purchases_5_pages_df.h5', 'df')

end4 = datetime.now()
print(end4 - start4)

0:00:09.626721


In [77]:
start4=datetime.now()

df_nonpurch = pd.read_hdf('df2_for_merge_nonpurchases_5_pages_df.h5', 'df')

end4 = datetime.now()
print(end4 - start4)

0:00:35.115068


##### Merge purchase and Non-Purchase Sessions

In [78]:
df_purch['purchase'] = True
df_nonpurch['purchase'] = False

In [79]:
df2 = df_purch.append(df_nonpurch, ignore_index=True)

C:\Users\Sellit\Anaconda3\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


In [80]:
# first row of every page of DataFrame 2
df2_head = df2.groupby(['id', 'startTime']).head(1)

In [81]:
del df_purch
del df_nonpurch
del df2

#### Load DataFrame 3

In [83]:
start4=datetime.now()

df3 = pd.read_hdf('DataFrame_3_for_merge_df.h5', 'df')

end4 = datetime.now()
print(end4 - start4)

0:00:00.073796


### Merge mean velocity, dist on page and time with no movement from DataFrame 2

In [84]:
df_out2 = pd.merge(df_out, df2_head[['id', 'startTime','mean_velocity', 'dist_on_page', 'time_no_movement']],left_on=['session_id', 'startTime'],right_on=['id', 'startTime'], how='left')

### Merge time on object aggregated from DataFrame 3

In [86]:
list_col1 = list(df3.columns[pd.Series(df3.columns).str.endswith('time_on_oject_aggregated')])
list_col2 = list(df3.columns[pd.Series(df3.columns).str.endswith('id')])

list_col1.extend(list_col2)

In [87]:
df_out3 = pd.merge(df_out2, df3[list_col1],left_on=['session_id'],right_on=['id'], how='left')

In [89]:
del df_out3['id_x']
del df_out3['id_y']

### Drop sessions that have a checkout in first five rows

In [90]:
df_out4 = df_out3.drop(df_out3[(df_out3['no_checkout_infirstrows'] == False)].index)

### Write file and save it to working directory

In [91]:
start2=datetime.now()

df_out4.to_hdf('DataFrame_1_five_pages_df_out.h5', 'df_out', mode='w')

end2 = datetime.now()
print(end2 - start2)

C:\Users\Sellit\Anaconda3\lib\site-packages\pandas\core\generic.py:1993: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block5_values] [items->['session_id', 'page_cat', 'uri', 'websitePage', 'ref_gen', 'page_id', 'tags1', 'title', 'country', 'region', 'city', 'isp', 'ip', 'lang', 'userAgent', 'browser', 'browserVersion', 'os', 'osVersion', 'device', 'referrer', 'referrerType', 'first_5_pages', 'screenRes', 'tags2', 'variables', 'lng', 'lat', 'annotations']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


0:00:00.605123
